# Hackaton : Fraud Bancaire

## Data Preparation (Feature Engineering)


## DATA PREPARATION / FEATURE ENGINEERING
### Auteurs : Hiba & Lisa


**Chargement des fichiers CSV et JSON**

In [ ]:
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

print(" Chargement des fichiers CSV et JSON...")

# CSV
transactions = pd.read_csv("transactions_train.csv")
cards = pd.read_csv("cards_data.csv")
users = pd.read_csv("users_data.csv")

# JSON : codes MCC
with open("mcc_codes.json") as f:
    mcc_codes = json.load(f)
mcc_df = pd.DataFrame(list(mcc_codes.items()), columns=["mcc", "mcc_description"])

# JSON : labels de fraude
with open("train_fraud_labels.json") as f:
    fraud_labels = json.load(f)

# Transformation du JSON de labels en DataFrame
fraud_df = pd.DataFrame(list(fraud_labels["target"].items()), columns=["transaction_id", "fraud"])
fraud_df["fraud"] = fraud_df["fraud"].map({"Yes": 1, "No": 0})

print("Chargement terminé !")


 Chargement des fichiers CSV et JSON...
Chargement terminé !


**APERÇU DES DONNÉES**

In [ ]:
print("\nTransactions : ", transactions.shape)
print("Cards        : ", cards.shape)
print("Users        : ", users.shape)
print("MCC codes    : ", mcc_df.shape)
print("Fraud labels : ", fraud_df.shape)

display(transactions.head(2))
display(cards.head(2))
display(users.head(2))
display(mcc_df.head(2))
display(fraud_df.head(2))



Transactions :  (210000, 12)
Cards        :  (6146, 13)
Users        :  (2000, 14)
MCC codes    :  (109, 2)
Fraud labels :  (210000, 2)


,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,5942,NaN
1,22160255,2018-11-26 17:44:00,1862,4049,$58.29,Chip Transaction,98648,Des Moines,IA,50313.0,5814,NaN


,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No


,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5


,mcc,mcc_description
0,5812,Eating Places and Restaurants
1,5541,Service Stations


,transaction_id,fraud
0,19765990,0
1,22160255,0


**NETTOYAGE DE DONNEES**

**1. Nettoyage du fichier transactions**

In [30]:
# nettoyage complet du fichier transactions avec remplacement de 'errors' par la référence MCC

import numpy as np


print("transactions - dimensions avant nettoyage :", transactions.shape)
print(transactions.info())

# suppression des doublons
transactions.drop_duplicates(inplace=True)

# nettoyage de la colonne 'amount' (retrait du signe $ et conversion en float)
transactions['amount'] = (
    transactions['amount']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)

# gestion des valeurs manquantes critiques
transactions = transactions.dropna(subset=['transaction_id', 'card_id'])
transactions['amount'] = transactions['amount'].fillna(transactions['amount'].median())

# correction des montants négatifs ou aberrants
transactions = transactions[transactions['amount'] >= 0]

# conversion de la colonne 'date' au format datetime
transactions['date'] = pd.to_datetime(transactions['date'], errors='coerce')

# suppression des lignes sans date valide
transactions = transactions.dropna(subset=['date'])

# --- remplacement de la colonne 'errors' par la référence MCC (à partir du fichier JSON) ---

# chargement du fichier mcc_codes.json
with open('mcc_codes.json', 'r') as f:
    mcc_data = json.load(f)

# gestion selon la structure du JSON
if isinstance(mcc_data, dict):
    # JSON sous forme de dictionnaire : { "5411": "Grocery stores", ... }
    mcc_ref = pd.DataFrame(list(mcc_data.items()), columns=['mcc', 'mcc_description'])
else:
    # JSON sous forme de liste d'objets : [ {"mcc": 5411, "mcc_description": "..."}, ... ]
    mcc_ref = pd.DataFrame(mcc_data)

# conversion du type pour assurer la correspondance
transactions['mcc'] = pd.to_numeric(transactions['mcc'], errors='coerce')
mcc_ref['mcc'] = pd.to_numeric(mcc_ref['mcc'], errors='coerce')

# fusion avec la référence MCC
transactions = transactions.merge(mcc_ref, on='mcc', how='left')

# suppression de la colonne 'errors' si elle existe
if 'errors' in transactions.columns:
    transactions.drop(columns=['errors'], inplace=True)
    print("colonne 'errors' supprimée et remplacée par 'mcc_description'.")

# gestion des valeurs manquantes dans la description MCC
transactions['mcc_description'] = transactions['mcc_description'].fillna('unknown')

# suppression des doublons finaux
transactions = transactions.drop_duplicates(subset=['transaction_id'])

# vérification finale
print("transactions - dimensions après nettoyage :", transactions.shape)
print(transactions.info())
print("\napercu des 5 premières lignes :")
display(transactions.head())


transactions - dimensions avant nettoyage : (199872, 12)
<class 'pandas.core.frame.DataFrame'>
Index: 199872 entries, 0 to 209999
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   transaction_id  199872 non-null  object        
 1   date            199872 non-null  datetime64[ns]
 2   client_id       199872 non-null  int64         
 3   card_id         199872 non-null  int64         
 4   amount          199872 non-null  float64       
 5   use_chip        199872 non-null  object        
 6   merchant_id     199872 non-null  int64         
 7   merchant_city   199872 non-null  object        
 8   merchant_state  174596 non-null  object        
 9   zip             173349 non-null  float64       
 10  mcc             199872 non-null  int64         
 11  errors          3108 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 19.8+ MB
None
colonne 

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,mcc_description
0,19765990,2017-07-07 09:53:00,1581,2519,12.35,Chip Transaction,20519,Arnold,MO,63010.0,5942,Book Stores
1,22160255,2018-11-26 17:44:00,1862,4049,58.29,Chip Transaction,98648,Des Moines,IA,50313.0,5814,Fast Food Restaurants
2,17566794,2016-03-26 12:42:00,1967,3367,11.03,Chip Transaction,46978,Lake Forest,CA,92630.0,5411,"Grocery Stores, Supermarkets"
3,17318690,2016-02-01 08:30:00,921,3457,85.74,Chip Transaction,63701,Rush,NY,14543.0,5411,"Grocery Stores, Supermarkets"
4,20994060,2018-03-24 14:42:00,456,2800,13.43,Chip Transaction,83271,Estero,FL,33928.0,4214,Motor Freight Carriers and Trucking


**2. Nettoyage du fichier cards**

In [28]:
# nettoyage complet du fichier cards

import pandas as pd
import numpy as np

print("\ncards - dimensions avant nettoyage :", cards.shape)
print(cards.info())

# remplacement des cellules vides ou composées uniquement d'espaces par NaN
cards = cards.replace(r'^\s*$', np.nan, regex=True)

# suppression des colonnes entièrement vides
cards = cards.dropna(axis=1, how='all')

# suppression des lignes entièrement vides
cards = cards.dropna(axis=0, how='all')

# suppression des doublons basés sur l'id de carte
cards.drop_duplicates(subset=['id'], inplace=True)

# nettoyage de la colonne 'credit_limit' (retrait du signe $ et conversion en float)
if 'credit_limit' in cards.columns:
    cards['credit_limit'] = (
        cards['credit_limit']
        .astype(str)
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
        .astype(float)
    )

# gestion des valeurs manquantes sur les variables catégorielles
for col in ['card_brand', 'card_type', 'has_chip']:
    if col in cards.columns:
        cards[col] = cards[col].fillna('unknown')

# gestion des valeurs manquantes sur les variables numériques
for col in ['num_cards_issued', 'credit_limit', 'year_pin_last_changed']:
    if col in cards.columns:
        cards[col] = pd.to_numeric(cards[col], errors='coerce')
        cards[col] = cards[col].fillna(cards[col].median())

# conversion des dates
if 'acct_open_date' in cards.columns:
    cards['acct_open_date'] = pd.to_datetime(cards['acct_open_date'], errors='coerce')

if 'expires' in cards.columns:
    # nettoyage du format d'expiration (MM/YY ou Mon-YY)
    cards['expires'] = pd.to_datetime(cards['expires'], errors='coerce', format='%b-%y') + pd.offsets.MonthEnd(0)

# suppression des cartes sans id ou client_id valide
cards = cards.dropna(subset=['id', 'client_id'])

# suppression des lignes avec valeurs critiques manquantes
cards = cards.dropna(subset=['credit_limit'])

# suppression des doublons restants
cards.drop_duplicates(inplace=True)

# suppression de la colonne sans intérêt 'card_on_dark_web' (valeur constante)
if 'card_on_dark_web' in cards.columns and cards['card_on_dark_web'].nunique() == 1:
    cards.drop(columns=['card_on_dark_web'], inplace=True)
    print("colonne 'card_on_dark_web' supprimée (valeur constante).")

# vérification finale
print("\ncards - dimensions après nettoyage :", cards.shape)
print(cards.info())
print("\napercu des 5 premières lignes :")
display(cards.head())




cards - dimensions avant nettoyage : (6146, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     6146 non-null   int64         
 1   client_id              6146 non-null   int64         
 2   card_brand             6146 non-null   object        
 3   card_type              6146 non-null   object        
 4   card_number            6146 non-null   int64         
 5   expires                6146 non-null   datetime64[ns]
 6   cvv                    6146 non-null   int64         
 7   has_chip               6146 non-null   object        
 8   num_cards_issued       6146 non-null   int64         
 9   credit_limit           6146 non-null   float64       
 10  acct_open_date         6146 non-null   datetime64[ns]
 11  year_pin_last_changed  6146 non-null   int64         
 12  card_on_dark_

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed
0,4524,825,Visa,Debit,4344676511950444,2022-12-31,623,YES,2,24295.0,2002-09-01,2008
1,2731,825,Visa,Debit,4956965974959986,2020-12-31,393,YES,2,21968.0,2014-04-01,2014
2,3701,825,Visa,Debit,4582313478255491,2024-02-29,719,YES,2,46414.0,2003-07-01,2004
3,42,825,Visa,Credit,4879494103069057,2024-08-31,693,NO,1,12400.0,2003-01-01,2012
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-31,75,YES,1,28.0,2008-09-01,2009


**3. Nettoyage du fichier users**

In [ ]:
print("\nusers - dimensions avant nettoyage :", users.shape)
print(users.info())

# suppression des doublons basés sur l'id utilisateur
users.drop_duplicates(subset=['id'], inplace=True)

# gestion des valeurs manquantes
users['gender'] = users['gender'].fillna('unknown')
users['address'] = users['address'].fillna('unknown')

# nettoyage des âges
users['current_age'] = pd.to_numeric(users['current_age'], errors='coerce')
users['retirement_age'] = pd.to_numeric(users['retirement_age'], errors='coerce')
users['birth_year'] = pd.to_numeric(users['birth_year'], errors='coerce')
users['birth_month'] = pd.to_numeric(users['birth_month'], errors='coerce')

# remplacement des valeurs manquantes d'âge par la médiane
users['current_age'] = users['current_age'].fillna(users['current_age'].median())
users['retirement_age'] = users['retirement_age'].fillna(users['retirement_age'].median())

# bornage raisonnable des âges
users = users[(users['current_age'] >= 18) & (users['current_age'] <= 100)]

# nettoyage des coordonnées géographiques
users['latitude'] = pd.to_numeric(users['latitude'], errors='coerce')
users['longitude'] = pd.to_numeric(users['longitude'], errors='coerce')

# nettoyage des valeurs financières (retrait du signe $ et conversion en float)
for col in ['per_capita_income', 'yearly_income', 'total_debt']:
    users[col] = (
        users[col]
        .astype(str)
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
        .astype(float)
    )

# gestion des valeurs manquantes pour les revenus et dettes
users['per_capita_income'] = users['per_capita_income'].fillna(users['per_capita_income'].median())
users['yearly_income'] = users['yearly_income'].fillna(users['yearly_income'].median())
users['total_debt'] = users['total_debt'].fillna(users['total_debt'].median())

# nettoyage du credit score
users['credit_score'] = pd.to_numeric(users['credit_score'], errors='coerce')
users['credit_score'] = users['credit_score'].fillna(users['credit_score'].median())

# nettoyage du nombre de cartes de crédit
users['num_credit_cards'] = pd.to_numeric(users['num_credit_cards'], errors='coerce').fillna(0).astype(int)

# suppression des lignes sans id valide
users = users.dropna(subset=['id'])

# vérification finale
users.drop_duplicates(subset=['id'], inplace=True)
print("users - dimensions après nettoyage :", users.shape)



users - dimensions avant nettoyage : (2000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month        2000 non-null   int64  
 5   gender             2000 non-null   object 
 6   address            2000 non-null   object 
 7   latitude           2000 non-null   float64
 8   longitude          2000 non-null   float64
 9   per_capita_income  2000 non-null   object 
 10  yearly_income      2000 non-null   object 
 11  total_debt         2000 non-null   object 
 12  credit_score       2000 non-null   int64  
 13  num_credit_cards   2000 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 218.9+ KB
None
users -

**Fusion finale des fichiers nettoyés (transactions, cartes, utilisateurs, mcc et fraude)**

In [ ]:
# fusion finale des fichiers nettoyés (transactions, cartes, utilisateurs, mcc et fraude)

# harmonisation du type de transaction_id avant la fusion
transactions['transaction_id'] = transactions['transaction_id'].astype(str)
fraud['transaction_id'] = fraud['transaction_id'].astype(str)

# fusion des transactions avec les labels de fraude
merged_df = transactions.merge(fraud, on='transaction_id', how='left')

# fusion avec les cartes
merged_df = merged_df.merge(
    cards,
    left_on='card_id',
    right_on='id',
    how='left',
    suffixes=('', '_card')
)

# fusion avec les utilisateurs
merged_df = merged_df.merge(
    users,
    left_on='client_id',
    right_on='id',
    how='left',
    suffixes=('', '_user')
)

# fusion avec les codes marchands
merged_df = merged_df.merge(mcc, on='mcc', how='left')

In [ ]:
# gestion des valeurs manquantes
merged_df['fraud'] = merged_df['fraud'].fillna(0).astype(int)
merged_df['mcc_description'] = merged_df['mcc_description'].fillna('unknown')

# vérification finale
print("forme finale du fichier fusionné :", merged_df.shape)
print(merged_df.info())

# sauvegarde du fichier fusionné
merged_df.to_csv('merged_dataset.csv', index=False)
print("fichier 'merged_dataset.csv' sauvegardé avec succès.")




forme finale du fichier fusionné : (199872, 41)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199872 entries, 0 to 199871
Data columns (total 41 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   transaction_id         199872 non-null  object        
 1   date                   199872 non-null  datetime64[ns]
 2   client_id              199872 non-null  int64         
 3   card_id                199872 non-null  int64         
 4   amount                 199872 non-null  float64       
 5   use_chip               199872 non-null  object        
 6   merchant_id            199872 non-null  int64         
 7   merchant_city          199872 non-null  object        
 8   merchant_state         174596 non-null  object        
 9   zip                    173349 non-null  float64       
 10  mcc                    199872 non-null  int64         
 11  errors                 3108 non-null    object        
 

In [37]:
# nettoyage final du dataset fusionné pour le machine learning et la soumission

import pandas as pd
import numpy as np

# lecture du dataset fusionné
df = pd.read_csv("merged_dataset.csv")
print("forme initiale :", df.shape)

# 1️⃣ remplacer les cellules vides ou espaces par NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# 2️⃣ suppression des colonnes entièrement vides
df = df.dropna(axis=1, how='all')

# 3️⃣ suppression des lignes entièrement vides
df = df.dropna(axis=0, how='all')

# 4️⃣ suppression des colonnes constantes (même valeur partout)
constant_cols = [col for col in df.columns if df[col].nunique(dropna=True) <= 1]
if constant_cols:
    print("colonnes supprimées (constantes) :", constant_cols)
    df = df.drop(columns=constant_cols)

# 5️⃣ suppression des colonnes identifiants ou redondantes inutiles pour le ML
id_like_cols = [
    'id', 'client_id', 'client_id_card', 'id_user',
    'card_number', 'cvv', 'zip', 'merchant_id',
    'birth_year', 'birth_month', 'address'
]
df = df.drop(columns=[c for c in id_like_cols if c in df.columns], errors='ignore')

# 6️⃣ suppression des colonnes textuelles non exploitables (trop libres)
text_cols = [
    'merchant_city', 'merchant_state', 'card_brand', 'card_type',
    'use_chip', 'has_chip', 'gender', 'mcc_description'
]
df = df.drop(columns=[c for c in text_cols if c in df.columns], errors='ignore')

# 7️⃣ suppression des colonnes contenant beaucoup de NaN (>80%)
nan_threshold = 0.8
cols_nan_ratio = df.isna().mean()
cols_to_drop_nan = cols_nan_ratio[cols_nan_ratio > nan_threshold].index
if len(cols_to_drop_nan) > 0:
    print("colonnes supprimées pour excès de valeurs manquantes :", list(cols_to_drop_nan))
    df.drop(columns=cols_to_drop_nan, inplace=True)

# 8️⃣ remplissage des NaN restants (numériques -> 0)
df = df.fillna(0)

# 9️⃣ conversion automatique des catégories ou booléens en codes numériques
for col in df.select_dtypes(include=['bool', 'object']).columns:
    if col != 'transaction_id':
        df[col] = df[col].astype('category').cat.codes

#  🔟 suppression des doublons
df.drop_duplicates(inplace=True)

# 11️⃣ sauvegarde du dataset final prêt pour le ML
df.to_csv("final_dataset_ml_ready.csv", index=False)
print("\n✅ fichier 'final_dataset_ml_ready.csv' sauvegardé avec succès.")
print("forme finale :", df.shape)

# 12️⃣ préparation du template de soumission
submission = df[['transaction_id', 'fraud']].copy()
submission.rename(columns={'fraud': 'fraud_prediction'}, inplace=True)
submission.to_csv("submission_template.csv", index=False)
print("✅ fichier 'submission_template.csv' sauvegardé avec succès (transaction_id + fraud_prediction).")

# aperçu final
print("\ncolonnes finales conservées :")
print(list(df.columns))
display(df.head())


forme initiale : (199872, 41)
colonnes supprimées (constantes) : ['card_on_dark_web']
colonnes supprimées pour excès de valeurs manquantes : ['errors']

✅ fichier 'final_dataset_ml_ready.csv' sauvegardé avec succès.
forme finale : (199872, 20)
✅ fichier 'submission_template.csv' sauvegardé avec succès (transaction_id + fraud_prediction).

colonnes finales conservées :
['transaction_id', 'date', 'card_id', 'amount', 'mcc', 'fraud', 'expires', 'num_cards_issued', 'credit_limit', 'acct_open_date', 'year_pin_last_changed', 'current_age', 'retirement_age', 'latitude', 'longitude', 'per_capita_income', 'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards']


,transaction_id,date,card_id,amount,mcc,fraud,expires,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,current_age,retirement_age,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,19765990,92584,2519,12.35,5942,0,58,1,26596.0,124,2015,38.0,65.0,38.42,-90.36,21744.0,44334.0,55173.0,684.0,5.0
1,22160255,177899,4049,58.29,5814,0,91,2,10200.0,247,2016,25.0,65.0,41.57,-93.61,18568.0,37864.0,57052.0,686.0,1.0
2,17566794,14044,3367,11.03,5411,0,55,2,38120.0,138,2011,55.0,74.0,33.64,-117.67,30307.0,61793.0,686.0,767.0,4.0
3,17318690,5183,3457,85.74,5411,0,51,1,35073.0,162,2009,74.0,66.0,42.98,-77.67,26072.0,54013.0,3628.0,779.0,8.0
4,20994060,136377,2800,13.43,4214,0,106,1,5800.0,31,2009,54.0,63.0,26.63,-81.99,17140.0,34947.0,49024.0,751.0,3.0


In [32]:
# nettoyage automatique du dataset fusionné pour la préparation finale

import pandas as pd
import numpy as np

# lecture du fichier fusionné
df = pd.read_csv("merged_dataset.csv")

print("forme initiale :", df.shape)

# remplacement des cellules vides ou composées uniquement d'espaces par NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# suppression des colonnes entièrement vides
df = df.dropna(axis=1, how='all')

# suppression des lignes entièrement vides
df = df.dropna(axis=0, how='all')

# suppression des colonnes constantes (toutes les valeurs identiques ou nulles)
constant_cols = [col for col in df.columns if df[col].nunique(dropna=True) <= 1]
if constant_cols:
    print("colonnes supprimées (constantes ou inutiles) :", constant_cols)
    df = df.drop(columns=constant_cols)

# suppression des colonnes qui ne peuvent pas être fusionnées ou interprétées (identifiants en double, texte trop libre)
# ici, on élimine tout ce qui n'est pas exploitable sans feature engineering
drop_types = ['object']
text_cols = [c for c in df.select_dtypes(include=drop_types).columns if c not in ['transaction_id']]
df = df.drop(columns=text_cols, errors='ignore')

# suppression des valeurs manquantes restantes
df = df.fillna(0)

# préparation du fichier final pour la soumission
# ici on garde transaction_id et fraud comme cibles
final = df[['transaction_id', 'fraud']].copy()
final.rename(columns={'fraud': 'fraud_prediction'}, inplace=True)

# suppression des lignes invalides
final = final.dropna(subset=['transaction_id'])
final['fraud_prediction'] = final['fraud_prediction'].astype(int)

# sauvegarde
final.to_csv("submission_ready.csv", index=False)

print("\n✅ fichier 'submission_ready.csv' sauvegardé avec succès.")
print("forme finale :", final.shape)
print("\ncolonnes restantes :", list(final.columns))
display(final.head())


forme initiale : (199872, 41)
colonnes supprimées (constantes ou inutiles) : ['card_on_dark_web']

✅ fichier 'submission_ready.csv' sauvegardé avec succès.
forme finale : (199872, 2)

colonnes restantes : ['transaction_id', 'fraud_prediction']


,transaction_id,fraud_prediction
0,19765990,0
1,22160255,0
2,17566794,0
3,17318690,0
4,20994060,0


**Feature Engineering**

In [41]:
# feature engineering avancé axé sur la détection de fraude

import numpy as np
import pandas as pd

# assurer la bonne conversion des dates
for col in ['date', 'acct_open_date', 'expires']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# 1️⃣ transformation du montant
df['log_amount'] = np.log1p(df['amount'])  # réduit l'effet des valeurs extrêmes
df['amount_zscore'] = (df['amount'] - df['amount'].mean()) / df['amount'].std()  # normalisation

# 2️⃣ caractéristiques temporelles
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['day_of_week'] = df['date'].dt.dayofweek

# périodes critiques (souvent plus de fraude)
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_night'] = df['hour'].isin([0, 1, 2, 3, 4, 5]).astype(int)
df['is_end_of_month'] = (df['day'] > 25).astype(int)

# 3️⃣ comportements transactionnels
df['amount_ratio_limit'] = (df['amount'] / (df['credit_limit'] + 1)).clip(0, 10)
df['is_high_spender'] = (df['amount_ratio_limit'] > 0.8).astype(int)

# 4️⃣ encodage MCC (fréquence d’utilisation)
mcc_freq = df['mcc'].value_counts(normalize=True)
df['mcc_encoded'] = df['mcc'].map(mcc_freq)

# 5️⃣ stabilité financière
df['debt_to_income'] = (df['total_debt'] / (df['yearly_income'] + 1)).clip(0, 10)
df['credit_utilization'] = df['credit_limit'] / (df['yearly_income'] + 1)
df['credit_risk_index'] = (1 - (df['credit_score'] / 850)) * df['amount_ratio_limit']

# 6️⃣ fourchettes d’âge et maturité financière
bins = [0, 25, 40, 60, 80, 120]
labels = ['18–25', '26–40', '41–60', '61–80', '80+']
df['age_group'] = pd.cut(df['current_age'], bins=bins, labels=labels, right=False)
df['age_group_encoded'] = df['age_group'].cat.codes

# proximité de la retraite
df['retirement_proximity'] = 1 - (df['current_age'] / (df['retirement_age'] + 1))
df['retirement_proximity'] = df['retirement_proximity'].clip(0, 1)

# 7️⃣ stabilité du compte
if 'acct_open_date' in df.columns:
    df['account_age_days'] = (pd.Timestamp.now() - df['acct_open_date']).dt.days.clip(lower=0).fillna(0)
if 'expires' in df.columns:
    df['days_to_expiry'] = (df['expires'] - pd.Timestamp.now()).dt.days.clip(lower=0).fillna(0)

# 8️⃣ synthèse d’un indice global de risque
df['fraud_risk_index'] = (
    0.3 * df['amount_ratio_limit'] +
    0.2 * df['credit_risk_index'] +
    0.15 * df['debt_to_income'] +
    0.1 * df['retirement_proximity'] +
    0.15 * df['is_night'] +
    0.1 * df['is_weekend']
).clip(0, 1)

# 9️⃣# 9️⃣ nettoyage final
# suppression des dates inutiles après extraction
df.drop(columns=['date', 'acct_open_date', 'expires'], inplace=True, errors='ignore')

# conversion de la colonne catégorielle en texte pour éviter le conflit
if 'age_group' in df.columns:
    df['age_group'] = df['age_group'].astype(str).replace('nan', 'unknown')

# remplissage des NaN restants pour les colonnes numériques uniquement
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(0)

print("✅ nettoyage final effectué sans erreur")


print("✅ feature engineering terminé")
print("nombre total de variables après transformation :", len(df.columns))
display(df.head())



✅ nettoyage final effectué sans erreur
✅ feature engineering terminé
nombre total de variables après transformation : 39


,transaction_id,card_id,amount,mcc,fraud,num_cards_issued,credit_limit,year_pin_last_changed,current_age,retirement_age,...,mcc_encoded,debt_to_income,credit_utilization,credit_risk_index,age_group,age_group_encoded,retirement_proximity,account_age_days,days_to_expiry,fraud_risk_index
0,19765990,2519,12.35,5942,0,1,26596.0,2015,38.0,65.0,...,0.018997,1.244457,0.599887,0.000091,26–40,1,0.424242,20398,0,0.379250
1,22160255,4049,58.29,5814,0,2,10200.0,2016,25.0,65.0,...,0.039250,1.506721,0.269378,0.001102,26–40,1,0.621212,20398,0,0.440064
2,17566794,3367,11.03,5411,0,2,38120.0,2011,55.0,74.0,...,0.128242,0.011101,0.616888,0.000028,41–60,2,0.266667,20398,0,0.178424
3,17318690,3457,85.74,5411,0,1,35073.0,2009,74.0,66.0,...,0.128242,0.067168,0.649332,0.000204,61–80,3,0.000000,20398,0,0.160849
4,20994060,2800,13.43,4214,0,1,5800.0,2009,54.0,63.0,...,0.008876,1.402770,0.165961,0.000270,41–60,2,0.156250,20398,0,0.376789


In [42]:
# sauvegarde du dataset après feature engineering

# conversion de la colonne catégorielle pour éviter les erreurs de typage
if 'age_group' in df.columns:
    df['age_group'] = df['age_group'].astype(str).replace('nan', 'unknown')

# remplissage des NaN restants pour les colonnes numériques
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(0)

# sauvegarde finale
df.to_csv("final_dataset_FE.csv", index=False)
print(" fichier 'final_dataset_FE.csv' sauvegardé avec succès.")
print("forme finale :", df.shape)


✅ fichier 'final_dataset_FE.csv' sauvegardé avec succès.
forme finale : (199872, 39)




---



**Phase 1 : Nettoyage des données**





| Étape                               | Action effectuée                                                         | Objectif                                          |
| ----------------------------------- | ------------------------------------------------------------------------ | ------------------------------------------------- |
| Suppression des doublons            | `drop_duplicates()` sur `transaction_id`, `card_id`, etc.                | Éviter les entrées multiples identiques           |
| Nettoyage des montants              | Retrait du `$`, conversion en `float`, suppression des valeurs négatives | Normaliser les données monétaires                 |
| Conversion des dates                | Transformation en format `datetime`                                      | Permettre les extractions temporelles             |
| Gestion des NaN                     | Remplacement ou suppression selon importance des colonnes                | Rendre les données exploitables                   |
| Suppression des colonnes inutiles   | `id`, `client_id_card`, `address`, `birth_year`, `birth_month`, etc.     | Enlever les identifiants et infos non pertinentes |
| Suppression des colonnes constantes | Colonnes avec une seule valeur (ex. `card_on_dark_web`) supprimées       | Éviter les variables sans variance                |
| Nettoyage des fichiers JSON         | Fusion propre des données `mcc` et `fraud`                               | Ajouter les descriptions et labels corrects       |
| Uniformisation des types            | Conversion cohérente (`int`, `float`, `str`)                             | Préparer pour la fusion et le ML                  |
| Fusion des fichiers                 | `transactions`, `cards`, `users`, `mcc`, `fraud`                         | Créer un seul dataset complet                     |


**Phase 2 : Feature Engineering (FE)**

---



| Nouvelle variable                             | Description                                  | Objectif / intérêt pour la fraude             |
| --------------------------------------------- | -------------------------------------------- | --------------------------------------------- |
| `log_amount`                                  | Logarithme du montant                        | Réduit l’impact des valeurs extrêmes          |
| `amount_zscore`                               | Score normalisé du montant                   | Détecte les transactions atypiques            |
| `hour`, `day`, `month`, `year`, `day_of_week` | Dérivées temporelles                         | Identifier des motifs de fraude dans le temps |
| `is_weekend`                                  | 1 si transaction le week-end                 | Plus de fraude hors jours ouvrés              |
| `is_night`                                    | 1 si transaction la nuit                     | Activité suspecte fréquente la nuit           |
| `is_end_of_month`                             | 1 si après le 25 du mois                     | Périodes de risque accru                      |
| `amount_ratio_limit`                          | Montant / limite de crédit                   | Dépenses disproportionnées = risque           |
| `is_high_spender`                             | 1 si ratio > 0.8                             | Indique un usage proche du plafond            |
| `mcc_encoded`                                 | Fréquence normalisée du code marchand        | Rare merchant → activité inhabituelle         |
| `debt_to_income`                              | Dette / revenu                               | Stress financier potentiel                    |
| `credit_utilization`                          | Crédit utilisé / revenu annuel               | Mesure de la dépendance au crédit             |
| `credit_risk_index`                           | Risque combiné (score + limite)              | Évalue la vulnérabilité financière            |
| `age_group_encoded`                           | Fourchettes d’âge encodées                   | Différencier les comportements par âge        |
| `retirement_proximity`                        | Proximité de la retraite                     | Détection d’anomalies avant retraite          |
| `account_age_days`                            | Ancienneté du compte                         | Nouveaux comptes = risque accru               |
| `days_to_expiry`                              | Temps avant expiration de la carte           | Activité suspecte sur cartes expirantes       |
| `fraud_risk_index`                            | Score synthétique basé sur plusieurs signaux | Combinaison comportementale et financière     |
